# Padding Oracle 
- When a decrypted CBC ciphertext ends in an invalid pad the web server returns a 403 error code (forbidden request). When the CBC padding is valid, but the message is malformed, the web server returns a 404 error code (URL not found).
```
http://crypto-class.appspot.com/po?er="your ciphertext here"
```
- The first ciphertext block is random IV,  the decrypted text block is ascii encoded
- the ciphertext following the `"po?er="` is a hex encoded AES CBC encryption with a random IV of some secret data about Alice's session.

In [1]:
import urllib3 as ul
from copy import deepcopy

In [2]:
ENDIAN = "big"

def xor(x, y, z):
    assert len(x) == len(y) == len(z)
    a = int.from_bytes(x, ENDIAN)
    b = int.from_bytes(y, ENDIAN)
    c = int.from_bytes(z, ENDIAN)
    r = a ^ b ^ c
    return r.to_bytes(len(x), ENDIAN)

BLOCKSIZE = 16

# Target: "http:domain.com/po?er="
class PaddingOracle:
    def __init__(self, target):
        self.target = target
        self.http = ul.PoolManager()
    
    # ct: string representing hex encoded 
    # 4 * 16 * 2 == 128 characters in length
    # 4 blocks of ciphertxt, 1 block IV, 3 blocks ciphertext
    def decrypt4blocks(self, ct, debug=True):
        
        assert len(ct) == 128
        assert self.status_query(ct) == 200

        iv, c0, c1, c2 = ct[:32], ct[32:64], ct[64:96], ct[96:]
        
        m0 = self.decrypt_block(c0, iv)
        print(" > ", m0)
        
        m1 = self.decrypt_block(c1, c0)
        print(" > ",  m1)
        
        # KNOWN ISSUE: 
        # m2 = self.decrypt_block(c2, c1)
        # print(" > ", m2)    
        # return str(m0 + m1 + m2, 'utf-8')
    
    def decrypt_block(self, c, c0_hex):
        m = bytearray(BLOCKSIZE)
        c0 = bytes.fromhex(c0_hex)
        
        for i in range(1, BLOCKSIZE + 1):
            n = bytes([i for _ in range(BLOCKSIZE)])
            self.overwrite_byte(m, c, i, n, c0)
        return m
    
    # Overwrites one byte in message m for each iteration
    def overwrite_byte(self, m, c, i, n, c0):
        CURRENT = BLOCKSIZE - i
        for g in range(256): # 0 - 255
            m[CURRENT] = g
            q = xor(n, m, c0).hex() + c
            if self.isnot403(q) is True:
                print(chr(g), end="")
                return
            
        print("Unable to find byte")
    
    def isnot403(self, q):
        r = self.http.request('GET', self.target + q, retries=False)
        return r.status != 403 
    
    def status_query(self, q):
        return self.http.request('GET', self.target + q, retries=False).status

In [3]:
TARGET = 'http://crypto-class.appspot.com/po?er='
CIPHERTEXT = "f20bdba6ff29eed7b046d1df9fb7000058b1ffb4210a580f748b4ac714c001bd4a61044426fb515dad3f21f18aa577c0bdf302936266926ff37dbf7035d5eeb4"

po = PaddingOracle(TARGET)
message = po.decrypt4blocks(CIPHERTEXT)
print(message)

 sdroW cigaM ehT >  bytearray(b'The Magic Words ')
sO hsimaeuqS era >  bytearray(b'are Squeamish Os')
None
